<a href="https://colab.research.google.com/github/MDankloff/Cluster-Bias-Disco/blob/main/Student_data_exploratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [19]:
students = pd.read_csv('students.csv')
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 35 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance                      4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Nacionality                                     4424 non-null   int64  
 7   Mother's qualification                          4424 non-null   int64  
 8   Father's qualification                          4424 non-null   int64  
 9   Mother's occupation                      

In [20]:
print(students['Target'].unique())

['Dropout' 'Graduate' 'Enrolled']


In [21]:
students = pd.get_dummies(students, columns=['Target'])
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance                      4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Nacionality                                     4424 non-null   int64  
 7   Mother's qualification                          4424 non-null   int64  
 8   Father's qualification                          4424 non-null   int64  
 9   Mother's occupation                      

In [23]:
x = students.drop(['Target_Dropout', 'Target_Enrolled', 'Target_Graduate'], axis = 1)
y1 = students['Target_Dropout']
y2 = students['Target_Enrolled']
y3 = students['Target_Graduate']

In [24]:
#Train on prediction drop out
x_train, x_test, y1_train, y1_test = train_test_split(x, y1, test_size=0.2, shuffle = True, stratify = y1)

model = RandomForestClassifier()
model.fit(x_train, y1_train)

y1hat = model.predict(x_test)
len(y1hat)

885

In [25]:
predictions = pd.DataFrame()
predictions['predicted_class'] = y1hat.tolist()
predictions = predictions.set_index(x_test.index)
predictions['true_class'] = y1_test

predictions['errors'] = abs(predictions['predicted_class'] - predictions['true_class'])
total_data = x_test.merge(predictions, how= 'outer', left_index=True, right_index=True)
errors = total_data['errors']
df_out = total_data.drop(['predicted_class', 'true_class', 'errors'], axis=1)
total_data[total_data['errors']==1]

#accuracy
accuracy = accuracy_score(predictions['true_class'], predictions['predicted_class'])
print(f"Accuracy: {accuracy:.4f}")

#confusion matrix
cm = confusion_matrix(predictions['true_class'], predictions['predicted_class'])
print(cm)

TypeError: numpy boolean subtract, the `-` operator, is not supported, use the bitwise_xor, the `^` operator, or the logical_xor function instead.